<a href="https://colab.research.google.com/github/tokyo8182/LLM-RAG/blob/main/Explore_knowledge_graph__and_graph_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount google drive to access dataset on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import python libraries that could be needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Install necessary langchain, groq and neo4j libraries. Langchain for LLM orchestration, groq
# for LLM and neo4j for graph database
!pip install -Uq langchain
!pip install -Uq langchain-community
!pip install -Uq langchain-groq
!pip install -Uq langchain_experimental
!pip install -Uq neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
## Credentials for accessing neo4j graph database. An instance of neo4j database
# needs to be created on-premise or on the cloud. For this notebook, cloud instance
# has been created whose credentials are below. To run this notebook independently,
# create a separate cloud instance of neo4j and use those credentials.
NEO4J_URI="neo4j+s://51dd2678.databases.neo4j.io:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="your neo4j api key"

In [ ]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [ ]:
#Instantiate an instance of neo4j graph database
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [ ]:
# API key for groq
groq_api_key="your groq api key"

In [ ]:
#Instantiate an instance of chatgroq (GROQ api). Use LLM model "Gemma2-9b-It"
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e8ee7f0f850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e8ee7f0d540>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# REad csv file (our dataset) into dataframe. Find total num of documents that's in the dataset. This assumes csv
# file is stored under /content/drive/MyDrive in google drive
df = pd.DataFrame
df = pd.read_csv("/content/drive/MyDrive/new_LK_tea_dataset_updated.csv")
len(df)

183

In [ ]:
#Identify unique classes that exist
df['class'].unique()

array(['ACT', 'Circulers', 'Guideline', 'Regulation'], dtype=object)

In [ ]:
# Find total number of documents that are of type/class 'Regulation'
df_regulation = df[df['class'] == 'Regulation']
len(df_regulation)

18

In [ ]:
# 'ACT'
df_act = df[df["class"] == 'ACT']
len(df_act)

35

In [ ]:
# 'Circular'
df_circular = df[df['class'] == 'Circulers']
len(df_circular)

120

In [ ]:
# 'Guideline'
df_guideline = df[df['class']== 'Guideline']
len(df_guideline)

10

In [ ]:
df_guideline[0:2]

,id,class,filename,path,url,language_label,required_ocr,has_tables,text_path,quality,data_origin,date_of_issuance,issuing_authority,topic,text_content
92,94,Guideline,TRISL_Guideline_04_2018_E.pdf,/content/drive/MyDrive/Omdena_Challenge/new_LK...,https://www.tri.lk/wp-content/uploads/2023/05/...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,01/11/2018,TRI,"Use of ""Soil Quality Index"" SQI for Decision M...",Page 1 of 3 \n TEA RESEARCH INSTITUTE OF SRI L...
93,95,Guideline,TRISL_Guideline_05_2018_E.pdf,/content/drive/MyDrive/Omdena_Challenge/new_LK...,https://www.tri.lk/wp-content/uploads/2023/05/...,en,no,no,/content/drive/MyDrive/Omdena_Challenge/new_LK...,good,scraped,01/12/2018,TRI,Guidelines on the Use of CO-3 Grass (Hybrid Na...,Page 1 of 3 \n TEA RESEARCH INSTITUTE OF SRI L...


In [ ]:
# print lengths of individual "Guideline" documents
for i in range(0,len(df_guideline)):
  print(i,len(df_guideline['text_content'].iloc[i]))

0 5665
1 7766
2 4629
3 5705
4 3621
5 4090
6 9823
7 8862
8 5686
9 8812


In [ ]:
# Consolidate the text in "Guideline"
text_guideline = ''
for i in range(0,len(df_guideline)):
  text_guideline += df_guideline['text_content'].iloc[i]

len(text_guideline)


64659

In [ ]:
text_guideline

"Page 1 of 3 \n TEA RESEARCH INSTITUTE OF SRI LANKA  \n \nIssued in: December 2018   Guideline No: 04/2018  \n \nGUIDELINES FOR ESTABLISHMENT OF ENERGY PLANTATIONS WITH  \n GLIRICIDIA SEPIUM AND CALLIANDRA CALO THRYSUS  \n \nIntroduction   \nTea processing requires a large quantity of fue lwood . Usually, it is estimated that 1 -\n1.5 kg of fuelwood is required to  process 1  kg of made tea. Additionally, plantation \nworkers also need  fuelwood for their domestic use.  Often, the total demand of \nfuelwood cannot be met within the plantation . In addition t o use of fuelwood for tea \nprocessing, it can be used as an energy source for generating electricity (Dendro \npower). As a result , there is a growing demand for fuelwood in Sri Lanka.  In order to \nensure regular and assured supply of fuelwood for industry req uirement, marginal tea \nfields  (low yielding)  and those abandoned owing to land suitability limitations (poor \nsoil conditions, rocky lands, excessive slopes , etc.) 

In [ ]:
# Transform the text for "Guideline" into knowledge graph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
import time

llm_transformer = LLMGraphTransformer(llm=llm)

# Split text_guideline into smaller chunks
chunk_size = 2500  # Adjust this value based on rate limits and token usage
text_chunks = [text_guideline[i:i + chunk_size] for i in range(0, len(text_guideline), chunk_size)]

graph_documents_guideline = []

# Process chunks with rate limiting
for chunk in text_chunks:
    documents = [Document(page_content=chunk)]
    try:
      graph_documents_guideline.extend(llm_transformer.convert_to_graph_documents(documents))
    except Exception as e:
      print(f"Error processing chunk: {e}")

    time.sleep(2) # Adjust the delay as needed to avoid rate limiting


Error processing chunk: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "DynamicGraph"\n\t\t},\n\t\t"parameters": {\n\t\t\t"nodes": [\n\t\t\t\t{\n\t\t\t\t\t"id": "TEA RESEARCH INSTITUTE OF SRI LANKA",\n\t\t\t\t\t"type": "organization"\n\t\t\t\t},\n\t\t\t\t{\n\t\t\t\t\t"id": "December 2018",\n\t\t\t\t\t"type": "date"\n\t\t\t\t},\n\t\t\t\t{\n\t\t\t\t\t"id": "04/2018",\n\t\t\t\t\t"type": "guideline_number"\n\t\t\t\t},\n\t\t\t\t{\n\t\t\t\t\t"id": "GUIDELINES FOR ESTABLISHMENT OF ENERGY PLANTATIONS WITH  GLIRICIDIA SEPIUM AND CALLIANDRA CALO THRYSUS",\n\t\t\t\t\t"type": "document"\n\t\t\t\t},\n\t\t\t\t{\n\t\t\t\t\t"id": "fuelwood",\n\t\t\t\t\t"type": "material"\n\t\t\t\t},\n\t\t\t\t{\n\t\t\t\t\t"id": "tea proc

In [ ]:
len(graph_documents_guideline)

24

# Look at different clusters in the knowledge graph

In [ ]:
graph_documents_guideline[0].nodes

[Node(id='Gliricidia', type='Plant', properties={}),
 Node(id='Calliandra', type='Plant', properties={})]

In [ ]:
graph_documents_guideline[0].relationships

[Relationship(source=Node(id='Gliricidia', type='Plant', properties={}), target=Node(id='Fuelwood', type='Resource', properties={}), type='PRODUCES', properties={}),
 Relationship(source=Node(id='Calliandra', type='Plant', properties={}), target=Node(id='Fuelwood', type='Resource', properties={}), type='PRODUCES', properties={})]

In [ ]:
graph_documents_guideline[12].nodes

[Node(id='Staff', type='Person', properties={}),
 Node(id='Safety Officer', type='Person', properties={}),
 Node(id='Community Members', type='Person', properties={})]

In [ ]:
graph_documents_guideline[12].relationships

[Relationship(source=Node(id='Staff', type='Person', properties={}), target=Node(id='Safety Officer', type='Person', properties={}), type='WORKS_WITH', properties={}),
 Relationship(source=Node(id='Staff', type='Person', properties={}), target=Node(id='Community Members', type='Person', properties={}), type='INTERACTS_WITH', properties={})]

In [ ]:
# Store the generated graph documents into a neo4j graph database
graph.add_graph_documents(
    graph_documents_guideline,
    baseEntityLabel= True,
    include_source=True)

In [ ]:
graph.refresh_schema()

In [ ]:
# Generate cypher query to query the knowledge graph
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7e8eeb8306d0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e8ee7f0f850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e8ee7f0d540>, model_name='Gemma2-9b-I

In [ ]:
# Ask a question
response = chain.invoke({"query" : "What organization is located in Talawakelle?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Organization)<-[:LOCATEDIN]-(l:Location) WHERE l.id='Talawakelle' RETURN o.id
Full Context:
[]

> Finished chain.


{'query': 'What organization is located in Talawakelle?',
 'result': "I don't know the answer. \n"}

In [ ]:
response = chain.invoke({"query" : "Where is Tea Research Institute Of Sri Lanka located in?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Organization {id: "Tea Research Institute Of Sri Lanka"})-[r:LOCATEDIN]->(l:Location) RETURN l.id
Full Context:
[{'l.id': 'Talawakelle'}]

> Finished chain.


{'query': 'Where is Tea Research Institute Of Sri Lanka located in?',
 'result': 'Talawakelle  \n'}

In [ ]:
response = chain.invoke({"query" : "What has Tea Research Institute Of Sri Lanka published?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MEMBER_OF]->(o:Organization {id: "Tea Research Institute Of Sri Lanka"})<-[:PUBLISHEDBY]-(d:Document)
RETURN d.id, d.text
Full Context:
[]

> Finished chain.


{'query': 'What has Tea Research Institute Of Sri Lanka published?',
 'result': "I don't know the answer. \n"}

In [ ]:
response = chain.invoke({"query" : "Who are the members of Vigilance committee?"})
response
#



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MEMBER_OF]->(o:Organization {name:"Vigilance committee"}) RETURN p.id 

Full Context:
[]

> Finished chain.


{'query': 'Who are the members of Vigilance committee?',
 'result': "I don't know the answer. \n"}

In [ ]:
response = chain.invoke({"query" : "What are the Interim Guidelines on Strategic Cost Component Management in Tea  Cultivation ?"})
response
#



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document {id:"Interim Guidelines on Strategic Cost Component Management in Tea Cultivation"}) RETURN d.text



Full Context:
[]

> Finished chain.


{'query': 'What are the Interim Guidelines on Strategic Cost Component Management in Tea  Cultivation ?',
 'result': "I don't know the answer. \n"}

In [ ]:
response = chain.invoke({"query" : "What does the document fc57899e05eeb02dc87cc90321278d92 say about best bonding clays ?"})
response
#



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document {id: 'fc57899e05eeb02dc87cc90321278d92'})
WHERE d.text CONTAINS 'best bonding clays'
RETURN d.text

Full Context:
[{'d.text': ' of AFS grain size 50  - 60 is generally  used.  \nb. Clay  \nThe best bonding clays are bentonites .   \nc. Coal dust  \nUsed mainly in iron foundries with so me being used in non -ferrous foundries.  \nd. BENTOKOL  \nBENTOKOL additives are blends of natural clays to which have been added \nspecially selected essential volatiles to provide both bond and volatile content .  \ne. Water  \nWater is needed to develop the clay bond bu t at times  it cause s casting defects.  \n \nTea Research Institute of Sri Lanka  \nTalawakelle  \nCOPYRIGHT  \nAll rights reserved. No parts of this publication may be reproduced or transmitted in any form or by \nany means, electronic or mechanical, including photocopying, recording or information storage and \nretrieval, without permission in writing 

{'query': 'What does the document fc57899e05eeb02dc87cc90321278d92 say about best bonding clays ?',
 'result': 'The best bonding clays are bentonites.  \n'}